In [10]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/ML VTU- RRCE/playtennis.csv')
df

,outlook,temperature,humidity,wind,class
0,0,0,0,0,0
1,0,0,0,1,0
2,1,0,0,0,1
3,2,1,0,0,1
4,2,2,1,0,1
5,2,2,1,1,0
6,1,2,1,1,1
7,0,1,0,0,0
8,0,2,1,0,1
9,2,1,1,0,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   outlook      16 non-null     int64
 1   temperature  16 non-null     int64
 2   humidity     16 non-null     int64
 3   wind         16 non-null     int64
 4   class        16 non-null     int64
dtypes: int64(5)
memory usage: 768.0 bytes


In [12]:
attributes = ('Outlook','Temperature','Humidity','Wind','PlayTennis')

In [13]:
def entropy(target_col):
  elements,counts = np.unique(target_col,return_counts = True)
  entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
  #print('Entropy =',entropy)
  return entropy

In [14]:
def InfoGain(data,split_attribute_name,target_name="class"):
  #Calculate the entropy of the total dataset
  total_entropy = entropy(data[target_name])
  #calculate entropy of the dataset 
  #Calculate the values and the corresponding counts for the split attribute 
  vals,counts = np.unique(data[split_attribute_name],return_counts=True)
  #Calculate the weighted entropy
  Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
  
  #Calculate the information gain
  Information_Gain = total_entropy - Weighted_Entropy
  return Information_Gain


In [15]:
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):
  #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
  #If all target_values have the same value, return this value
  if len(np.unique(data[target_attribute_name])) <= 1:
    return np.unique(data[target_attribute_name])[0]
  elif len(data)==0:
    return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
  elif len(features) ==0:
    return parent_node_class  
  #If none of the above holds true, grow the tree!
  else:
    #Set the default value for this node --> The mode target feature value of the current node
    parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]   
    #Select the feature which best splits the dataset

  item_values = [InfoGain(data,feature,target_attribute_name) for feature in features]
  #Return the information gain values for the features in the dataset
  best_feature_index = np.argmax(item_values)
  best_feature = features[best_feature_index]
  #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information gain in the first run
  tree = {best_feature:{}}
  features = [i for i in features if i != best_feature]
  #Remove the feature with the best inforamtion gain from the feature space

#Grow a branch under the root node for each possible value of the root node feature
  for value in np.unique(data[best_feature]):
    value = value
#Split the dataset along the value of the feature with the largest information gain and there with create sub_datasets
    sub_data = data.where(data[best_feature] == value).dropna()
#Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
    subtree = ID3(sub_data,df,features,target_attribute_name,parent_node_class)
#Add the sub tree, grown from the sub_dataset to the tree under the root node
    tree[best_feature][value] = subtree
  return(tree)


In [16]:
def predict(query,tree,default = 1):
  #1.
  for key in list(query.keys()):
    if key in list(tree.keys()):
      #2.
      try:
        result = tree[key][query[key]]
      except:
        return default
 #3.
      result = tree[key][query[key]]
 #4.
      if isinstance(result,dict):
        return predict(query,result)
      else:
        return result

#We drop the index respectively relabel the index
#starting form 0, because we do not want to run into errors regarding the row labels /index #testing_data = dataset.iloc[10:].reset_index(drop=True)


def train_test_split(dataset):
  training_data = dataset.iloc[:14].reset_index(drop=True)
  return training_data

In [17]:
#,testing_data
def test(data,tree):
  queries = data.iloc[:,:-1].to_dict(orient = "records")
  predicted = pd.DataFrame(columns=["predicted"]) 
  #Calculate the prediction accuracy
  for i in range(len(data)):
    predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 


In [18]:
XX = train_test_split(df)
training_data=XX
#testing_data=XX[1]
tree = ID3(training_data,training_data,training_data.columns[:-1])
print('Display Tree',tree)
print('len=',len(training_data))
test(training_data,tree)

Display Tree {'outlook': {0: {'humidity': {0.0: 0.0, 1.0: 1.0}}, 1: 1.0, 2: {'wind': {0.0: 1.0, 1.0: 0.0}}}}
len= 14
